In [ ]:
# Import the load_dotenv function
from dotenv import load_dotenv
import os
# Load the .env file
load_dotenv('../.env')  # Assuming your .env file is in the same directory

# Example: Accessing an environment variable
openai_access_key = os.getenv('OPENAI_ACCESS_KEY')
huggingface_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')
cohere_api_token = os.getenv('COHERE_API_KEY')

In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Criteria {i+1}:\n\nPage Content: {d.page_content}\nNCT ID: {d.metadata.get('nct_id', 'N/A')}\nCriteria Type: {d.metadata.get('criteria_type', 'N/A')}" for i, d in enumerate(docs)]
        )
    )

In [ ]:
import json
import os
from langchain.docstore.document import Document
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

In [ ]:
# from transformers import pipeline
# classifier = pipeline("zero-shot-classification")
# candidate_labels = ["negative", "neutral"]

# Specify the directory containing your JSON files
json_directory = '../../data/trials_jsons/'
desired_fields = ["nct_id", "eligibility"]

# Initialize an empty list to store the loaded data
docs = []

# Loop through each file in the directory
for filename in os.listdir(json_directory):
    if filename.endswith('.json'):
        # Construct the full path to the JSON file
        file_path = os.path.join(json_directory, filename)
        # Open and load the JSON file
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            extracted_data = {field: json_data.get(field) for field in desired_fields}

            # Extracting eligibility criteria
            eligibility_criteria = json_data.get("eligibility")
            if eligibility_criteria is not None:
                for index, criterion in enumerate(eligibility_criteria):
                    # Creating metadata for each criterion
                    metadata = {
                        "nct_id" :extracted_data['nct_id'],
                        "idx": index +1,
                    }
                    # Adding the "field" value to metadata
                    metadata["criteria_type"] = criterion["entities_data"][0]["field"]  # Assuming field is same for all entities
                    # Adding entities data to metadata with indexes
                    for i, entity in enumerate(criterion["entities_data"]):
                        for key, value in entity.items():
                            if key != "field":
                                metadata[f"{key}_{i + 1}"] = value

                    # Creating a document for each criterion
                    doc = Document(page_content=criterion["text"], metadata=metadata)
                    docs.append(doc)
                    
vectorstore = Chroma.from_documents(docs, SentenceTransformerEmbeddings(), persist_directory="../../data/db/", collection_name="criteria")

In [ ]:
vectorstore.persist()
vectorstore = None

In [ ]:
import json
import os
from langchain.docstore.document import Document

# Specify the directory containing your JSON files
json_directory = '../../data/trials_jsons/'
desired_fields = ["nct_id", "brief_title", "brief_summary", "condition", "gender", "minimum_age", "maximum_age", "phase"]
fields_to_concatenate = ["brief_title", "brief_summary"]

# Initialize an empty list to store the loaded data
docs = []
ids = []

# Loop through each file in the directory
for filename in os.listdir(json_directory):
    if filename.endswith('.json'):
        # Construct the full path to the JSON file
        file_path = os.path.join(json_directory, filename)

        # Open and load the JSON file
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            extracted_data = {field: json_data.get(field) for field in desired_fields}
            ids.append(extracted_data["nct_id"])
            
            # Construct metadata, handling None values
            metadata = {
                "id": extracted_data.get("nct_id", ""),
                "condition": extracted_data.get("condition", ""),
                "gender": extracted_data.get("gender", ""),
                "minimum_age": extracted_data.get("minimum_age", ""),
                "maximum_age": extracted_data.get("maximum_age", ""),
                "phase": extracted_data.get("phase", ""),
            }
            # Remove None values from metadata
            metadata = {k: v for k, v in metadata.items() if v is not None}
                
            concatenated_string = ', '.join(str(extracted_data[field]) for field in fields_to_concatenate)
            docs.append(Document(page_content=concatenated_string, metadata=metadata))


In [ ]:
vectorstore = Chroma.from_documents(docs, SentenceTransformerEmbeddings(), persist_directory="../../data/db", collection_name="trials")

In [ ]:
db3 = Chroma(persist_directory="../../data/db", embedding_function= SentenceTransformerEmbeddings(), collection_name="criteria")
retriever = db3.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.5, "k":1500},
    filters=None,
)

In [ ]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of clinical trial eligibility criteria records"""

    queries: List[str] = Field(
        ...,
        description="Distinct queries to search for",
    )

In [ ]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

output_parser = PydanticToolsParser(tools=[Search])

system = """
You are tasked with a critical role: to dissect a complex, structured query into its component sub-queries. Each component of the query is encapsulated in a JSON dictionary, representing a unique aspect of the information sought. Your objective is to meticulously parse this JSON, isolating each field as a standalone sub-query. These sub-queries are the keys to unlocking detailed, specific information pertinent to each field.

As you embark on this task, remember:
- Treat each JSON field with precision, extracting it as an individual query without altering its essence.
- Your analysis should preserve the integrity of each sub-query, ensuring that the original context and purpose remain intact.
- Enhance each sub-query by contextually expanding it into a complete, meaningful sentence. The aim is to transform each piece of data into a narrative that provides insight into the patient's health condition or medical history.
- Approach this task with the understanding that the fidelity of the sub-queries to their source is paramount. Alterations or misinterpretations could lead to inaccuracies in the information retrieved.

This meticulous separation of the structured query into clear, unmodified sub-queries is fundamental. It enables a tailored search for information, enhancing the relevance and accuracy of the responses generated. Your role in this process is not just to parse data, but to ensure that each piece of information extracted is a faithful reflection of the query's intent, ready to be matched with precise and relevant data points.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=openai_access_key)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [ ]:
from langchain_core.runnables import chain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


@chain
async def custom_chain(question):
    response = await query_analyzer.ainvoke(question)
    docs = []
    for query in response.queries:
        new_docs = await compression_retriever.ainvoke(query)
        docs.extend(new_docs)
    # You probably want to think about reranking or deduplicating documents here
    # But that is a separate topic
    return docs

In [ ]:
import json
f  = open("../../data/synthetic_patients/1234.json")
query = str(json.load(f))
docs_result = await custom_chain.ainvoke(query)

In [ ]:
import openai

openai.api_key = openai_access_key

def rerank_answers(question, candidate_answers):
    scored_answers = []

    for answer in candidate_answers:
        prompt = f"Question: {question}\nAnswer: {answer}\n\nHow relevant and correct is this answer to the question above? Rate from 1 (least relevant) to 10 (most relevant)."
        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            max_tokens=3,  # We only need a short numeric response
            n=1
        )
        score = int(response['choices'][0]['text'].strip())
        scored_answers.append((answer, score))

    # Sort answers based on the score
    scored_answers.sort(key=lambda x: x[1], reverse=True)  # Higher scores first

    return scored_answers


In [ ]:
question = query
candidate_answers = docs_result[0:3]

scored_answers = rerank_answers(question, candidate_answers)
for answer, score in scored_answers:
    print(f"Score: {score}, Answer: {answer}")


In [ ]:
query_analyzer.invoke(query) 

In [ ]:
pretty_print_docs(docs_result)
# docs_result[0].metadata["nct_id"]

In [ ]:
# get a new token: https://dashboard.cohere.ai/

import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

llm = Cohere(temperature=0)
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

In [ ]:
@chain
async def custom_reranker_chain(question):
    response = await query_analyzer.ainvoke(question)
    docs = []
    for query in response.queries:
        new_docs = await compression_retriever.ainvoke(query)
        docs.extend(new_docs)
    return docs

In [ ]:
await custom_reranker_chain("Patient has diabetes | Patient has COVID-19 | Colorectal Cancer Patient with KRAS mutation | Patient has a diagnosis of COVID-19")

In [ ]:
from langchain.aggregators import SimpleAggregator

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

async def refine_query_with_llm(query):
    """
    Uses an LLM to expand and refine the query to make it more specific to clinical trial criteria.
    
    Args:
    query (str): The initial query to be refined.
    
    Returns:
    str: The refined and expanded query.
    """
    # Define a prompt that instructs the LLM on how to expand the query
    prompt = (
        f"Given a patient profile for a clinical trial, refine and expand the following query to be more specific and contextual:\n\n"
        f"Query: {query}\n\n"
        "Refined Query:"
    )
    
    # Use LangChain's structured LLM interaction method
    # Note: Adjust the method invocation based on the specific LangChain version and LLM interface you are using
    response = await llm.invoke(prompt=prompt, max_tokens=100, temperature=0.7, stop_sequences=["\n"])
    refined_query = response['choices'][0]['text'].strip()
    
    # Fallback in case the LLM does not generate a useful output
    if not refined_query:
        refined_query = query
    
    return refined_query


In [ ]:
g = await refine_query_with_llm("Stage III colon adenocarcinoma")

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/nli-deberta-v3-large')
scores = model.predict([('The patient has KRAS mutation', 'The man has cancer')])

#Convert scores to labels
label_mapping = ['disagreement', 'agreement', 'neutral']
labels = [label_mapping[score_max] for score_max in scores.argmax(axis=1)]


In [1]:
from crossencoder_reranker import *

/home/mabdallah/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model = model_fn("")
transform_fn(model, "{\"pairs\": [[\"Patient with KRAS mutation\", \"Patient without KRAS mutation\"], [\"Patient with KRAS mutation\", \"KRAS mutation positive\"]]}", "application/json", "application/json")


'{"scores": [-2.5739340782165527, 4.65949010848999]}'